In [1]:
# !pip install --upgrade gdown
# !gdown 1-4elQY1C-n23u3QqomnLiI9CN9iPrWC3
# !tar -xzf genres.tar.gz

In [2]:
from pathlib import Path

data_dir = Path("genres")
assert data_dir.exists()

In [3]:
wav_fns = list(data_dir.rglob("*.wav")) # rglob: recursive glob
len(wav_fns)

1000

In [4]:
import torchaudio

wav_fn = wav_fns[0]
y, sr = torchaudio.load(wav_fn)

import IPython.display as ipd
# print(y.shape, sr)
# ipd.Audio(y, rate=sr)

# Dataset

In [5]:
new_sr = 16000
resampler = torchaudio.transforms.Resample(sr, new_sr)
y_resampled = resampler(y)

In [6]:
from tqdm import tqdm
import random
random.seed(42)

class Dataset:
    def __init__(self, data_dir, target_sr=16000, is_test=False):
        self.data_dir = Path(data_dir)
        self.wav_fns = list(self.data_dir.rglob("*.wav"))
        # rglob itself is a generator -> make it as a list

        self.origin_freq = 22050
        self.target_sr = target_sr
        self.resampler = torchaudio.transforms.Resample(self.origin_freq, self.target_sr)
        
        self.is_test = is_test

        self.audio_dur = 2 # TODO: get this as argument

        self.audio_label_pairs = self.load_audio_label_pairs()
        self.class_names = self.make_class_vocab()
        

    def load_audio_label_pairs(self):
        audio_label_pairs = []
        selected_fns = self.wav_fns[:800] if not self.is_test else self.wav_fns[800:]
        for wav_fn in tqdm(selected_fns):
            y, sr = torchaudio.load(wav_fn)
            assert sr == self.origin_freq, f"Expected {self.origin_freq} but got {sr}"
            y = self.resampler(y)
            audio_label_pairs.append((y, wav_fn.parent.name))
            # parent.name: genres/blues/blues.00000.wav -> blues
            # use paired audio and label, to prevent the mismatch
        return audio_label_pairs

    def make_class_vocab(self):
        class_names = [label for _, label in self.audio_label_pairs]
        class_names = sorted(list(set(class_names)))
        return class_names

    # dataset class에 꼭 필요한 두 가지: __len__, __getitem__
    def __len__(self):
        return len(self.audio_label_pairs)
    
    def __getitem__(self, idx):
        audio, label = self.audio_label_pairs[idx]

        audio_len = audio.shape[1]
        max_end_point = audio_len - self.target_sr * self.audio_dur
        start_point = random.randint(0, max_end_point - 1)
        audio = audio[:, start_point:start_point + self.target_sr * self.audio_dur]

        return audio, self.class_names.index(label)

dataset = Dataset('genres')

100%|██████████| 800/800 [00:41<00:00, 19.50it/s]


In [7]:
dataset.class_names

['blues', 'classical', 'country', 'hiphop', 'jazz', 'metal', 'reggae', 'rock']

In [8]:
# Check whether audio shape are all the same
min(set([audio.shape[1] for audio, _ in dataset.audio_label_pairs])), 16000*30

(478912, 480000)

## torch.utils.data.DataLoader
- 자동으로 dataset의 getitem을 호출해서 하나의 텐서로 묶어줌
    - 텐서로 변환하는 함수: collate_fn
    - 매 배치마다 어떤 idx로 dataset getitem을 호출할지 자동으로 결정
    - shuffle=True면 자동으로 셔플링 된 샘플 호출
    - 매 epoch마다 shuffle 순서 초기화

In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in train_loader:
    break
audios, labels = batch
audios.shape, labels.shape

(torch.Size([32, 1, 32000]), torch.Size([32]))

## Make Model
- Mel Spectrogram Converter
- nn.Linear?

In [11]:
import torch
import torch.nn as nn

class MelDbConverter(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.mel_conv = torchaudio.transforms.MelSpectrogram(n_fft=1024,
                                                         hop_length=512,
                                                         f_min=20,
                                                         f_max=4000,
                                                         n_mels=80,
                                                         sample_rate=16000)
    self.db_conv = torchaudio.transforms.AmplitudeToDB()

  def forward(self, x):
    return self.db_conv(self.mel_conv(x))


class Model(nn.Module):
    def __init__(self, dim=128, num_out = 10):
        super().__init__() # init nn.Module

        self.spec = MelDbConverter()
        self.layers = nn.Sequential(
           nn.Linear(5040, dim),
           nn.ReLU(),
           nn.Linear(dim, dim),
           nn.ReLU(),
          nn.Linear(dim, 10)
        )

    def forward(self, x):
        spec = self.spec(x)
        flattened = spec.reshape(spec.shape[0], -1)
        out = self.layers(flattened)
        
        return self.spec(x)
    
model = Model()
model(audios).shape

torch.Size([32, 1, 80, 63])

### Spec Converter
Audio -> spectrogram으로 옮기는 과정이
1) model에 들어갈 수도 있고
2) dataset 처리의 일부로 처리할 수도 있음
- 이 경우, audio -> spectrogram으로 변환하는 데 필요한 n_fft, hop_size 등등의 내용까지 모두 동일하게 설정해야 모델이 올바르게 작동하게 됨
- 예를 들어 f_max 설정이 원래와 달라졌음에도 불구하고 spectrogram의 shape은 동일해서, 돌아가긴 함에도 불구하고 이상한 결과가 나오는 상황이 발생할 수도.

### 따라서
- spec converter도 모델의 일부로 둬야 한다!!

### model.state_dict(), model.parameters()
- spec converter와 관련된 건 state_dict에는 포함
    - model.state_dict().keys()를 찍어보면 spec converter와 관련된 것들이 있음
- however! model.named_parameters()[0] 이렇게 학습되는 parameter의 이름들을 뽑아보면, spec converter에 관한 건 없음
- 이렇게 모델에는 있지만 학습되는 건 아닌 부분은 'buffer'라고 부른다
- 모델 내부에 buffer를 두면 모델을 gpu에 올릴 때 이 부분도 동일한 device로 가게 되어서 편리함!

### Calculate Number of Parameters

In [16]:
for name, param in model.named_parameters():
    print(name, param.shape, param.numel())

    """
    Result:
    layers.0.weight torch.Size([128, 5040]) 645120
    layers.0.bias torch.Size([128]) 128
    layers.2.weight torch.Size([128, 128]) 16384
    layers.2.bias torch.Size([128]) 128
    layers.4.weight torch.Size([10, 128]) 1280
    layers.4.bias torch.Size([10]) 10

    중간에 있는 ReLU는 학습되지 않기 때문에 여기 없음
    """

layers.0.weight torch.Size([128, 5040]) 645120
layers.0.bias torch.Size([128]) 128
layers.2.weight torch.Size([128, 128]) 16384
layers.2.bias torch.Size([128]) 128
layers.4.weight torch.Size([10, 128]) 1280
layers.4.bias torch.Size([10]) 10


### Test/Validation Set
- test 할 때마다 random하게 자르는 거 하면 안 된다 (train과 다르게)
- testset을 고정 시켜놓아서, 결과가 달라지는 것이 온전히 모델에 의한 것이도록 해야 함. testset의 randomness에 영향을 받는 게 아닌.

## Get Test Accuracy

In [17]:
# For validation or test:
# use larger batch size (it is okay since calculation cost is smaller than training)
# Do not shuffle
test_loader = DataLoader(Dataset('genres', is_test=True), batch_size=200, shuffle=False, drop_last=False)
# Drop Last:
# Training 할 때는 쓸 수도 있음
# Test/Validation: Do not drop last.

100%|██████████| 200/200 [00:09<00:00, 20.28it/s]


In [19]:
model.eval() # change dropout, batchnorm, etc. to evaluation mode
model.to('cuda')
with torch.inference_mode(): # do not need to calculate gradient
    # torch.no_grad() is about not updating the gradient,
    # but torch.inference_mode() is about not calculating the gradient so it is slightly faster

    total_loss = []
    for batch in test_loader:
        audios, labels = batch
        prob = model(audios.to('cuda'))
        loss = -torch.log(prob[torch.arange(len(labels)), labels])
        total_loss.append(loss)

### TODO(minigb): Train the model

### Get Particular Index of the tensor
- torch.argmax
- torch.argsort: sort한 결과에서의 idx 구하기

### Loss 확인하기
- model의 결과를 들어보면서 실험 하기
    - loss가 높은 거 top5
    - loss가 낮은 거 top5
    - 이런 식으로 어떤 example을 잘 맞추고, 어떤 걸 잘 못 맞추는지

# CNN
- 현재 spectrogram shape: (32, 80, 63)
    - 지금은 channel이 없는 상태
    - conv layer가 sweep 하는 동안 input에서 같은 위치의 여러 channel이 모두 확인된다.
- spec.unsqueeze(1): add empty dimension on 1st axis -> shape은 (32, 1, 80, 63). 채널 추가

In [23]:
# unsqueeze(1)로 in channel을 하나 만들었다.
kernel = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3)
# stride: 1, padding: 0 by default
kernel.weight.shape

# M) in_channel과 out_channel이 좀 헷갈렸는데


torch.Size([2, 1, 3, 3])

# Notes
- duration 더 길게 해서 보면 더 높은 accuracy.
- 전반적으로 더 높은 confidence를 가지게 됨
- 그래서 loss가 낮은 test set을 확인해보면 그 loss는 이전보다 더 커짐
- 전반적으로 overfitting 되는 것도 있고